In [19]:
# Training loop:
# 1. Create torch dataset from the data containing the query, positive, negative example.
# 2. Create a dataloader from the dataset.
# 3. Run a single batch through the model.

In [20]:
!poetry add pandas

The following packages are already present in the pyproject.toml and will be skipped:

  • pandas

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.


In [19]:
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
import pickle

In [12]:
## Load data
train = pd.read_parquet("train.parquet")
test = pd.read_parquet("test.parquet")
validate = pd.read_parquet("validate.parquet")

In [23]:
print(train.shape, test.shape, validate.shape)

(82326, 6) (9650, 6) (10047, 6)


In [16]:
full_dataset = pd.concat([train, test, validate])

In [17]:
full_dataset[full_dataset["query_id"].isin([19699, 19701])]

,answers,passages,query,query_id,query_type,wellFormedAnswers
0,[Results-Based Accountability is a disciplined...,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...",what is rba,19699,description,[]
2,[20-25 minutes],"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...",how long do you need for sydney and surroundin...,19701,numeric,[]


In [ ]:
# Save the Query IDs for each dataset to use on later.

In [13]:
total_size = train.shape[0] + test.shape[0] + validate.shape[0]
print(total_size)
# Rerun this code only if you want to generate new indices.

102023


In [14]:
# # Unpickle and test
# with open("indices.pkl", "rb") as f:
#     all_indices = pickle.load(f)

#     print(f"{len(all_indices['train'])} training indices {all_indices['train'][0:10]}")
#     print(f"{len(all_indices['test'])} test indices {all_indices['test'][0:10]}")
#     print(
#         f"{len(all_indices['validate'])} validate indices {all_indices['validate'][0:10]}"
#     )

with open("query_ids.pkl", "rb") as f:
    all_query_ids = pickle.load(f)
    print(
        f"{len(all_query_ids['train'])} training query ids {all_query_ids['train'][0:10]}"
    )
    print(f"{len(all_query_ids['test'])} test query ids {all_query_ids['test'][0:10]}")
    print(
        f"{len(all_query_ids['validate'])} validate query ids {all_query_ids['validate'][0:10]}"
    )

82326 training query ids [19699, 19700, 19701, 19702, 19703, 19704, 19705, 19706, 19707, 19708]
9650 test query ids [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
10047 validate query ids [9652, 9653, 9654, 9655, 9656, 9657, 9658, 9659, 9660, 9661]


In [20]:
# train[['passages', 'query']].iloc[0]
train_dataset = full_dataset[full_dataset.query_id.isin(all_query_ids["train"])]
test_dataset = full_dataset[full_dataset.query_id.isin(all_query_ids["test"])]
validate_dataset = full_dataset[full_dataset.query_id.isin(all_query_ids["validate"])]

train_dataset.head()["query"].iloc[0]

'what is rba'

In [21]:
print(f"{np.all(train.query_id.value_counts() == 1)}")
print(f"{np.all(train_dataset.query_id.value_counts() == 1)}")
assert np.all(train.query_id.value_counts() == 1)
assert np.all(train_dataset.query_id.value_counts() == 1)
assert np.all(validate.query_id.value_counts() == 1)

True
True


In [238]:
full_dataset.head()

,answers,passages,query,query_id,query_type,wellFormedAnswers
0,[Results-Based Accountability is a disciplined...,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...",what is rba,19699,description,[]
1,[Yes],"{'is_selected': [0, 1, 0, 0, 0, 0, 0], 'passag...",was ronald reagan a democrat,19700,description,[]
2,[20-25 minutes],"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...",how long do you need for sydney and surroundin...,19701,numeric,[]
3,[$11 to $22 per square foot],"{'is_selected': [0, 0, 0, 0, 0, 0, 0, 0, 1], '...",price to install tile in shower,19702,numeric,[]
4,[Due to symptoms in the body],"{'is_selected': [0, 0, 1, 0, 0, 0, 0, 0], 'pas...",why conversion observed in body,19703,description,[]


In [22]:
from utils.data_utils import create_lookups, add_hashed_urls

In [33]:
ids_to_urls, urls_to_ids, url_to_doctext_mapping = create_lookups(full_dataset)
# add_hashed_urls(train_dataset, urls_to_ids)
# add_hashed_urls(test_dataset, urls_to_ids)
# add_hashed_urls(validate_dataset, urls_to_ids)

dataset shape: (102023, 6)
Total number of urls: 837729
Total number of unique urls: 456487
Total number of hashed urls: 456487
Total number of queries: 102023


In [35]:
with open("url_to_doctext_mapping.pkl", "wb") as f:
    pickle.dump(url_to_doctext_mapping, f)

In [36]:
with open("url_to_doctext_mapping.pkl", "rb") as f:
    url_to_doctext_mapping = pickle.load(f)

In [365]:
print(url_to_doctext_mapping["51456f6eae52e682e8cc2877123547df"])
print(urls_to_ids["http://rba-africa.com/about/what-is-rba/"])
print(ids_to_urls["c2f05ed2ec73e1b97799c70e4cdbed8f"])

For example, you could contribute $2,500 to a Roth IRA with one provider and $3,000 with a second provider. You cannot contribute $5,500 to the first Roth IRA and $5,500 to the second provider. Your total contribution limit is $5,500 for all accounts for the tax year. Individuals may contribute money to a Roth IRA from January 1 of the current tax year to April 15 of the following tax year. Contributions are made with post-tax dollars. You pay tax today so that in retirement no tax is charged to you in retirement.
c2f05ed2ec73e1b97799c70e4cdbed8f
http://rba-africa.com/about/what-is-rba/


In [ ]:
from tqdm import tqdm

tqdm.pandas()

In [366]:
master_url_id_set = np.array(
    list(set(ids_to_urls.keys()))
)  # Define or load your master_url_id_set here

# Triples

In [131]:
# Splitting the dataset for demonstration; in practice, adjust based on your dataset size
from concurrent.futures import ProcessPoolExecutor, as_completed
from functools import partial
import utils
import importlib

importlib.reload(utils)
importlib.reload(utils.data_utils)
from utils.data_utils import add_negative_samples

In [367]:
training_dataset_with_negative_samples = train_dataset.copy()
add_negative_samples(training_dataset_with_negative_samples ids_to_urls)

/Users/sid/workspace/mlx-week3/.venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
Processing Batches: 100%|██████████| 10/10 [11:02<00:00, 66.24s/it]  


In [368]:
assert np.all(training_dataset_with_negative_samples.query_id.value_counts() == 1)

In [370]:
test_dataset_with_negative_samples = test_dataset.copy()
add_negative_samples(test_dataset_with_negative_samples, ids_to_urls)

/Users/sid/workspace/mlx-week3/.venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
Processing Batches: 100%|██████████| 10/10 [01:18<00:00,  7.88s/it]


In [371]:
# Sanity check
assert np.all(test_dataset_with_negative_samples.query_id.value_counts() == 1)

In [372]:
validate_dataset_with_negative_samples = validate_dataset.copy()
add_negative_samples(validate_dataset_with_negative_samples, ids_to_urls)

/Users/sid/workspace/mlx-week3/.venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
Processing Batches: 100%|██████████| 10/10 [01:29<00:00,  8.98s/it]


In [373]:
# Sanity check
assert np.all(validate_dataset_with_negative_samples.query_id.value_counts() == 1)

In [410]:
training_dataset_copy[triple_columns].head()

,query_id,query,hashed_urls,negative_sample_urls
0,19699,what is rba,"[58b2b8024507a2126d53e59a87820ad8, affa5fde381...","[a7509f1dc6a82e1058cac78d284eee49, 9673999d279..."
1,19700,was ronald reagan a democrat,"[a2d179699e02966bdf9faa65b992daab, f097b1f42e4...","[06aa259b817892b5290b4b037c376591, 79b11837191..."
2,19701,how long do you need for sydney and surroundin...,"[70185a71a8f71d8d1029a71419d5cabf, f8e326ab289...","[6c364273217b774565d18c413d7b0095, 4e96bb1cbd3..."
3,19702,price to install tile in shower,"[49abbb4ce58adb0395a9ca4e1846239c, 0a1266e71ec...","[75dc8659a62b8bff44831a44f18463aa, 70d696bab58..."
4,19703,why conversion observed in body,"[de25b567b657116e9617dabbf554ac44, ea0e0b397cf...","[bf10f48ebac0dd6ece976b659f25d061, 2606c7238d4..."


In [374]:
triple_columns = ["query_id", "query", "hashed_urls", "negative_sample_urls"]
training_dataset_with_negative_samples[triple_columns].to_parquet(
    "training_dataset_triplets.parquet", engine="pyarrow"
)

In [375]:
test_dataset_with_negative_samples[triple_columns].to_parquet(
    "test_dataset_triplets.parquet", engine="pyarrow"
)

In [376]:
validate_dataset_with_negative_samples[triple_columns].to_parquet(
    "validate_dataset_triplets.parquet", engine="pyarrow"
)

In [47]:
training_dataset_with_negative_samples = pd.read_parquet(
    "training_dataset_triplets.parquet", engine="pyarrow"
)

In [45]:
test_dataset_with_negative_samples = pd.read_parquet(
    "test_dataset_triplets.parquet", engine="pyarrow"
)

In [43]:
validate_dataset_with_negative_samples = pd.read_parquet(
    "validate_dataset_triplets.parquet", engine="pyarrow"
)

In [44]:
validate_dataset_with_negative_samples.head()

,query_id,query,hashed_urls,negative_sample_urls
0,9652,walgreens store sales average,"[1433e7121275f4a258259851c76c58a6, 54504bcc083...","[365827554c81d1d2aa92af8b2ef64865, e05ad7e8f0b..."
1,9653,how much do bartenders make,"[3d93ed295e97e2251f59927ac4d7be07, 018cb526487...","[f5fae9bf50b51b495fb651a2e260d530, feba625ac8c..."
2,9654,what is a furuncle boil,"[311040d01c190045b8224c7bb183ddd1, fd92f401d6a...","[6c364273217b774565d18c413d7b0095, 66f0cd88114..."
3,9655,what can urinalysis detect,"[7cba506e3d46b6548c5af5511a96e51d, 5030d15e411...","[67e91a1ed2be6fe7744cf9351cf5f903, b38be53d559..."
4,9656,what is vitamin a used for,"[a6ef2e3eadc1a7c8658cb4d1b7c9d16e, c1a5800e031...","[4935f2c416b1670552b6374fcf502cd7, 1995b56c86a..."


In [377]:
# There is a unit test to test this in the tests/ dir.
s1 = list(
    [
        len(i[1]["negative_sample_urls"])
        for i in training_dataset_with_negative_samples.iterrows()
    ]
)
s2 = list(
    [
        len(i[1]["hashed_urls"])
        for i in training_dataset_with_negative_samples.iterrows()
    ]
)
print(len(s1))
print(len(s2))
print(s1[0:10])
print(s2[0:10])
assert s1 == s2

82326
82326
[7, 6, 6, 6, 7, 8, 4, 6, 7, 5]
[7, 6, 6, 6, 7, 8, 4, 6, 7, 5]


# Triples -> Embeddings


In [159]:
import sentencepiece as spm
from gensim.models import Word2Vec
import multiprocessing

In [160]:
sp = spm.SentencePieceProcessor()
sp.load("spm_AllTexts.model")

True

In [60]:
EMBEDDING_SIZE = 128

In [162]:
w2v_model = Word2Vec(
    min_count=20,
    window=10,
    vector_size=EMBEDDING_SIZE,
    sample=6e-5,
    alpha=0.03,
    min_alpha=0.0007,
    negative=20,
    workers=multiprocessing.cpu_count() - 1,
)

In [163]:
w2v_model = Word2Vec.load("word2vec.model")

### Convert the tripleID's to embeddings

In [499]:
import importlib

importlib.reload(utils)
importlib.reload(utils.data_utils)
from utils.data_utils import triples_to_embeddings

In [500]:
training_triples = triples_to_embeddings(
    training_dataset_with_negative_samples[triple_columns],
    url_to_doctext_mapping,
    sp,
    w2v_model,
    embedding_size=EMBEDDING_SIZE,
)

  0%|          | 0/82326 [00:00<?, ?it/s]

There are 7 emebddings with [44, 70, 103, 146, 147, 60, 145]


ValueError: expected sequence of length 44 at dim 1 (got 70)

In [458]:
print(len(training_triples[0][2]))
print(len(training_triples[0][3]))

7
7


In [487]:
testing_triples = triples_to_embeddings(
    test_dataset_with_negative_samples[triple_columns],
    url_to_doctext_mapping,
    sp,
    w2v_model,
    embedding_size=EMBEDDING_SIZE,
)

100%|██████████| 9650/9650 [02:43<00:00, 59.18it/s]


In [ ]:
testing_triples[0].query_embedding

In [380]:
validation_triples = triples_to_embeddings(
    validate_dataset_with_negative_samples[triple_columns],
    url_to_doctext_mapping,
    sp,
    w2v_model,
    embedding_size=EMBEDDING_SIZE,
)

100%|██████████| 10047/10047 [00:21<00:00, 471.36it/s]


In [412]:
# Pickle these to use for training
import pickle

with open("training_triples.pkl", "wb") as f:
    pickle.dump([tuple(i) for i in training_triples], f)
with open("testing_triples.pkl", "wb") as f:
    pickle.dump([tuple(i) for i in testing_triples], f)
with open("validation_triples.pkl", "wb") as f:
    pickle.dump([tuple(i) for i in validation_triples], f)

### Prepare Data.

In [7]:
import pickle

with open("training_triples.pkl", "rb") as f:
    training_triples = pickle.load(f)
with open("testing_triples.pkl", "rb") as f:
    testing_triples = pickle.load(f)
with open("validation_triples.pkl", "rb") as f:
    validation_triples_t = pickle.load(f)

In [74]:
len(training_triples)

82326

In [29]:
# training_dataset_copy[(training_dataset_copy.query_id == 55966)]
np.all(train.query_id.value_counts().values == 1)
train_dataset.query_id.value_counts()

query_id
19699     1
74586     1
74644     1
74643     1
74642     1
         ..
47164     1
47163     1
47162     1
47161     1
102128    1
Name: count, Length: 82326, dtype: int64

In [26]:
train_dataset.head()

,answers,passages,query,query_id,query_type,wellFormedAnswers
0,[Results-Based Accountability is a disciplined...,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...",what is rba,19699,description,[]
1,[Yes],"{'is_selected': [0, 1, 0, 0, 0, 0, 0], 'passag...",was ronald reagan a democrat,19700,description,[]
2,[20-25 minutes],"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...",how long do you need for sydney and surroundin...,19701,numeric,[]
3,[$11 to $22 per square foot],"{'is_selected': [0, 0, 0, 0, 0, 0, 0, 0, 1], '...",price to install tile in shower,19702,numeric,[]
4,[Due to symptoms in the body],"{'is_selected': [0, 0, 1, 0, 0, 0, 0, 0], 'pas...",why conversion observed in body,19703,description,[]


In [42]:
training_triples[0]

(19699,
 array([-0.45027733, -1.1089361 , -0.7485446 ,  0.29338717, -1.1459185 ,
        -1.5272202 , -0.74012953, -0.30964288,  0.21130872,  0.35100073,
         1.0060655 ,  0.20887893, -0.24708146, -0.731051  , -1.4602807 ,
         0.34196234,  0.1258754 , -0.38644987,  0.4168012 , -0.5980486 ,
         0.4608552 , -0.68917155,  0.09706956, -0.32858768, -2.4480584 ,
        -0.6501933 ,  0.6384356 , -2.113722  ,  0.46189746,  1.6031395 ,
        -0.7843428 , -0.94217765,  0.07016011,  0.06641492, -2.471242  ,
         1.3342896 ,  0.41738454, -1.3223062 , -3.1056533 , -0.44302684,
         1.8227277 , -0.5422932 ,  1.686431  ,  0.74762636, -0.5201043 ,
         0.24003752, -2.5744867 , -2.4474618 ,  0.31771022, -1.8747869 ,
        -0.329535  , -1.818454  ,  0.7101862 , -1.9354166 ,  0.5667974 ,
         0.419285  , -1.0025371 , -1.5425253 ,  2.3222175 ,  1.239261  ,
         0.09211183, -0.49247086, -1.3927358 , -0.1261586 ,  1.4260572 ,
        -1.3440863 ,  0.27712378, -0.728670

In [49]:
training_dataset_with_negative_samples.head()

,query_id,query,hashed_urls,negative_sample_urls
0,19699,what is rba,"[58b2b8024507a2126d53e59a87820ad8, affa5fde381...","[a7509f1dc6a82e1058cac78d284eee49, 9673999d279..."
1,19700,was ronald reagan a democrat,"[a2d179699e02966bdf9faa65b992daab, f097b1f42e4...","[06aa259b817892b5290b4b037c376591, 79b11837191..."
2,19701,how long do you need for sydney and surroundin...,"[70185a71a8f71d8d1029a71419d5cabf, f8e326ab289...","[6c364273217b774565d18c413d7b0095, 4e96bb1cbd3..."
3,19702,price to install tile in shower,"[49abbb4ce58adb0395a9ca4e1846239c, 0a1266e71ec...","[75dc8659a62b8bff44831a44f18463aa, 70d696bab58..."
4,19703,why conversion observed in body,"[de25b567b657116e9617dabbf554ac44, ea0e0b397cf...","[bf10f48ebac0dd6ece976b659f25d061, 2606c7238d4..."


In [54]:
from utils.data_utils import Triple

for i, training_triple in enumerate(training_triples[0:5]):
    training_triple = Triple(*training_triple)
    print(f"{i}th triple with query_id: {training_triple[0]}")
    print(
        f"\t#Relevant Docs = {len(training_triple.relevant_doc_embedding)}, Irrelevant Docs = {len(training_triple.irrelevant_doc_embedding)}"
    )
    print(
        f"\tAll are of shaped: {training_triple.relevant_doc_embedding[0].shape}? {all([i.shape == training_triple.relevant_doc_embedding[0].shape for i in training_triple.relevant_doc_embedding])}"
    )
    print(
        f"\tThe query text is {training_dataset_with_negative_samples[training_dataset_with_negative_samples.query_id == training_triple.query_id]['query'].values}"
    )

    relevant_text = train_dataset[train_dataset.query_id == training_triple[0]][
        "passages"
    ].values[0]["passage_text"]
    irrelavant_doc_hashids = training_dataset_with_negative_samples[
        training_dataset_with_negative_samples.query_id == training_triple[0]
    ]["negative_sample_urls"].values[0]
    irrelavant_text = [url_to_doctext_mapping[_id] for _id in irrelavant_doc_hashids]
    # relevant_text = [url_to_doctext_mapping[i] for i in training_triple.relevant_doc_embedding]
    # irrelavant_text = [url_to_doctext_mapping[i] for i in training_triple.irrelevant_doc_embedding]

    # id -> URL ->
    # The relevan
    NL = "\n"
    TAB = "\t"
    print(
        f"\tHere is the relevant docs, { (NL+TAB+TAB).join([str(i)[0:100] + '...' for i in relevant_text])}"
    )
    print(
        f"\t Here are the irrelevant docs, { (NL+TAB+TAB).join([str(i)[0:100] + '...' for i in irrelavant_text])}"
    )

0th triple with query_id: 19699
	#Relevant Docs = 7, Irrelevant Docs = 7
	All are of shaped: (128,)? True
	The query text is ['what is rba']
	Here is the relevant docs, Since 2007, the RBA's outstanding reputation has been affected by the 'Securency' or NPA scandal. Th...
		The Reserve Bank of Australia (RBA) came into being on 14 January 1960 as Australia 's central bank ...
		RBA Recognized with the 2014 Microsoft US Regional Partner of the ... by PR Newswire. Contract Award...
		The inner workings of a rebuildable atomizer are surprisingly simple. The coil inside the RBA is mad...
		Results-Based Accountability® (also known as RBA) is a disciplined way of thinking and taking action...
		Results-Based Accountability® (also known as RBA) is a disciplined way of thinking and taking action...
		RBA uses a data-driven, decision-making process to help communities and organizations get beyond tal...
		vs. NetIQ Identity Manager. Risk-based authentication (RBA) is a method of applying varyi

In [402]:
irrelavant_text

["Overview. As with most legendary Pokemon, Celebi is blessed with great base stats all around. It also has a good and synergetic typing, an awesome ability that lets it act as a status absorber, and a phenomenal movepool. These traits allow Celebi to play many roles, depending on your team's needs. Heatran is possibly the best Celebi partner out there as it has almost flawless defensive synergy with it, with Rock being the only type that the two don't resist together, and also beats many Pokemon that can set up on Celebi or OHKO it, such as Scizor, Forretress, Ferrothorn, and Skarmory.",
 'Calcium Carbonate (Limestone) Calcium carbonate, the chief component of limestone, is a widely used amendment to neutralize soil acidity and to supply calcium (Ca) for plant nutrition. The term “lime” can refer to several products, but for agricultural use it generally refers to ground limestone. ',
 'Health Benefits of Minerals. Below is a list of some of the minerals found in the body, including t

# RNN Definition 

In [471]:
import torch

torch.tensor(training_triples[0][2]).shape

torch.Size([7, 128])

In [485]:
for i in range(3):
    print(
        "Query is at index 1 and has Embedding dimension", len(training_triples[i][1])
    )
    print(
        "# Relevant document is len=", len(training_triples[i].relevant_doc_embedding)
    )
    print(
        "Each relevant doc embedding is len=",
        len(training_triples[i].relevant_doc_embedding[0]),
    )
    print(
        "# Irrelvant docs are the same for each relevant doc= ",
        len(training_triples[i].irrelevant_doc_embedding),
    )
    print(
        "# these are too having embedding dimension=",
        len(training_triples[0].irrelevant_doc_embedding[0]),
    )

Query is at index 1 and has Embedding dimension 128
Relevant document is len= 7
Each relevant doc embedding is len= 128
# Irrelvant docs are the same for each relevant doc=  7
# these are too having embedding dimension= 128
Query is at index 1 and has Embedding dimension 128
Relevant document is len= 6
Each relevant doc embedding is len= 128
# Irrelvant docs are the same for each relevant doc=  6
# these are too having embedding dimension= 128
Query is at index 1 and has Embedding dimension 128
Relevant document is len= 6
Each relevant doc embedding is len= 128
# Irrelvant docs are the same for each relevant doc=  6
# these are too having embedding dimension= 128


In [61]:
# Feed the data through two tower RNN, simple RNNs to
# start. No LSTM or GRUs with a margin and triplet loss.
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np


def triplet_loss(anchor, positive, negative, margin=1.0):
    print(anchor.shape, positive.shape, negative.shape)
    distance_positive = (anchor - positive).pow(2).sum(1)
    distance_negative = (anchor - negative).pow(2).sum(1)
    losses = torch.relu(distance_positive - distance_negative + margin)
    return losses.mean()

In [64]:
# Assuming D is the dimensionality of your embeddings
# input_size = EMBEDDING_SIZE  # Dimensionality of the input
hidden_size = 256  # Size of the RNN's hidden layer
output_size = 64  # Size of the final outout

rnn_doc = nn.RNN(EMBEDDING_SIZE, hidden_size)
rnn_answers = nn.RNN(EMBEDDING_SIZE, hidden_size)
linear = nn.Linear(hidden_size * 3, output_size)
#

### NEXT: Training loop for the rnn



In [65]:
optimizer = optim.Adam(
    list(rnn_doc.parameters()) + list(rnn_answers.parameters()), lr=0.001
)

In [67]:
training_triples[0]

(19699,
 array([-0.45027733, -1.1089361 , -0.7485446 ,  0.29338717, -1.1459185 ,
        -1.5272202 , -0.74012953, -0.30964288,  0.21130872,  0.35100073,
         1.0060655 ,  0.20887893, -0.24708146, -0.731051  , -1.4602807 ,
         0.34196234,  0.1258754 , -0.38644987,  0.4168012 , -0.5980486 ,
         0.4608552 , -0.68917155,  0.09706956, -0.32858768, -2.4480584 ,
        -0.6501933 ,  0.6384356 , -2.113722  ,  0.46189746,  1.6031395 ,
        -0.7843428 , -0.94217765,  0.07016011,  0.06641492, -2.471242  ,
         1.3342896 ,  0.41738454, -1.3223062 , -3.1056533 , -0.44302684,
         1.8227277 , -0.5422932 ,  1.686431  ,  0.74762636, -0.5201043 ,
         0.24003752, -2.5744867 , -2.4474618 ,  0.31771022, -1.8747869 ,
        -0.329535  , -1.818454  ,  0.7101862 , -1.9354166 ,  0.5667974 ,
         0.419285  , -1.0025371 , -1.5425253 ,  2.3222175 ,  1.239261  ,
         0.09211183, -0.49247086, -1.3927358 , -0.1261586 ,  1.4260572 ,
        -1.3440863 ,  0.27712378, -0.728670

In [71]:
for epoch in range(1):
    for i, (
        _,
        query_embedding,
        relevant_docs_embeddings,
        irrelevant_docs_embeddings,
    ) in enumerate(training_triples):
        optimizer.zero_grad()
        # print(query_embedding.shape)
        # print(len(relevant_docs_embeddings))
        # print(irrelevant_docs_embeddings[0].shape)
        # print('.')
        # RUn the relevant examples through the rnn_answer
        # Run the query through the rnn_doc
        # Calculate the loss
        # Backpropagate
        # Optimize
        # query_output, _ = rnn_doc(1, 1, query_embedding)
        losses = []
        # stack them as T*B*E, where T is sequence length, B = 1, E = embedding size
        relevant_docs_embeddings_tensor = (
            torch.tensor(relevant_docs_embeddings)
            .clone()
            .detach()
            .requires_grad_(False)
            .view(len(relevant_docs_embeddings), 1, -1)
        )
        # rd_view = relevant_docs_embeddings.view(len(relevant_docs_embeddings), 1, -1)
        # print(rd_view.shape)
        irrelevant_docs_embeddings_tensor = (
            torch.tensor(irrelevant_docs_embeddings)
            .clone()
            .detach()
            .requires_grad_(False)
            .view(len(irrelevant_docs_embeddings), 1, -1)
        )
        # print(ird_view.shape)
        query_embedding_tensor = (
            torch.tensor(query_embedding)
            .clone()
            .detach()
            .requires_grad_(False)
            .view(1, 1, -1)
        )
        query_output, _ = rnn_doc(query_embedding_tensor)
        pos_doc_output, _ = rnn_answers(relevant_docs_embeddings_tensor)
        neg_doc_output, _ = rnn_answers(irrelevant_docs_embeddings_tensor)
        print(query_output.shape, pos_doc_output.shape, neg_doc_output.shape)
        # 1. I want to horizontally stack the quer_outoput pos_doc_output and neg_doc_output doc
        # 2. pass it through a linear layer
        
        loss = triplet_loss(query_output, pos_doc_output, neg_doc_output)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()

        # for pos_doc_emb, neg_doc_emb in zip(relevant_docs_embeddings, irrelevant_docs_embeddings):
        #     pos_doc_output = rnn_answers(pos_doc_emb.unsqueeze(0).unsqueeze(0))  # Adjust dimensions if necessary
        #     neg_doc_output = doc_rnn(neg_doc_emb.unsqueeze(0))  # Adjust dimensions if necessary
        # loss = triplet_loss(query_output, pos_doc_output, neg_doc_output)
        # losses.append(loss)

        if i > 1:
            break
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {np.mean(losses)}")

torch.Size([1, 1, 256]) torch.Size([7, 1, 256]) torch.Size([7, 1, 256])
torch.Size([1, 1, 256]) torch.Size([7, 1, 256]) torch.Size([7, 1, 256])
torch.Size([1, 1, 256]) torch.Size([6, 1, 256]) torch.Size([6, 1, 256])
torch.Size([1, 1, 256]) torch.Size([6, 1, 256]) torch.Size([6, 1, 256])
torch.Size([1, 1, 256]) torch.Size([6, 1, 256]) torch.Size([6, 1, 256])
torch.Size([1, 1, 256]) torch.Size([6, 1, 256]) torch.Size([6, 1, 256])
torch.Size([1, 1, 256]) torch.Size([6, 1, 256]) torch.Size([6, 1, 256])
torch.Size([1, 1, 256]) torch.Size([6, 1, 256]) torch.Size([6, 1, 256])
torch.Size([1, 1, 256]) torch.Size([7, 1, 256]) torch.Size([7, 1, 256])
torch.Size([1, 1, 256]) torch.Size([7, 1, 256]) torch.Size([7, 1, 256])
torch.Size([1, 1, 256]) torch.Size([8, 1, 256]) torch.Size([8, 1, 256])
torch.Size([1, 1, 256]) torch.Size([8, 1, 256]) torch.Size([8, 1, 256])
torch.Size([1, 1, 256]) torch.Size([4, 1, 256]) torch.Size([4, 1, 256])
torch.Size([1, 1, 256]) torch.Size([4, 1, 256]) torch.Size([4, 1

In [73]:
relevant_docs_embeddings_tensor.shape

torch.Size([8, 1, 128])

In [524]:
inputs = [torch.randn(1, 4) for _ in range(5)]

In [514]:
for i in inputs:
    print(".")
    print(i)
    print(i.view(1, 1, -1))

.
tensor([[1.2411, 0.6259, 1.1159]])
tensor([[[1.2411, 0.6259, 1.1159]]])
.
tensor([[0.6223, 1.2001, 0.7883]])
tensor([[[0.6223, 1.2001, 0.7883]]])
.
tensor([[ 0.6911, -1.6744,  0.8264]])
tensor([[[ 0.6911, -1.6744,  0.8264]]])
.
tensor([[-0.0656, -1.2043,  0.6886]])
tensor([[[-0.0656, -1.2043,  0.6886]]])
.
tensor([[0.3123, 0.6284, 0.6254]])
tensor([[[0.3123, 0.6284, 0.6254]]])


In [515]:
inputs = torch.cat(inputs).view(len(inputs), 1, -1)

In [525]:
torch.cat(inputs).view(len(inputs), 1, -1)
# [
#  [[1, 2, 3]],
#     [[4, 5, 6]],
#     [[7, 8, 9]],
#     [[10, 11, 12]],
#     [[13, 14, 15]]
# ]

tensor([[[-0.8364, -0.7349,  0.2562,  1.7806]],

        [[-0.1919, -1.2498,  0.9293, -0.8176]],

        [[-0.1253, -1.9675, -0.9701,  0.7632]],

        [[-0.1750,  0.2369,  0.0190, -0.4398]],

        [[-1.0326, -0.0065,  1.7244, -0.4539]]])

In [526]:
e = nn.Embedding(5, 10)

In [531]:
e(torch.tensor([1, 2, 3])).shape  # 3*10, 3 tokens

torch.Size([3, 10])

In [586]:
print(rnn.weight_hh_l0)
print(rnn.weight_ih_l0)

Parameter containing:
tensor([[-0.2045,  0.1789,  0.1087,  0.2622],
        [ 0.2944, -0.0556,  0.3093, -0.2797],
        [ 0.2557,  0.0878, -0.3043,  0.2969],
        [-0.1137, -0.3761,  0.1893,  0.0344]], requires_grad=True)
Parameter containing:
tensor([[ 0.0807, -0.0887,  0.4171],
        [-0.3953,  0.2120, -0.1455],
        [ 0.2667,  0.4024, -0.0607],
        [-0.1381,  0.3133,  0.2092]], requires_grad=True)


In [585]:
rnn = nn.RNN(3, 4)  # Input dim is 3, output dim is 3
data = [
    [0.1, 0.2, 0.3],
    [-1, 0, 1],
    [-1, 1, 1],
    [0, 0.1, 0.2],
    [-0.5, 0, 0.5],
    [0.1, 0.2, 0.3],
    [-1, 0, 1],
    [-1, 1, 1],
    [0, 0.1, 0.2],
    [-0.5, 0, 0.5],
]
inputs = [
    torch.tensor(row, dtype=torch.float) for row in data
]  # make a sequence of length 5

# initialize the hidden state.
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = rnn(i.view(1, 1, -1), torch.randn(1, 1, 4))

# inputs_stacked = torch.cat(inputs).view(len(inputs), 1, -1)
inputs_stacked = torch.cat(inputs).view(5, 2, -1)
# hidden = (torch.randn(1, 2, 3), torch.randn(1, 2, 3))  # clean out hidden state
out, hidden = rnn(inputs_stacked, torch.randn(1, 2, 4))
print(out)
print(hidden)

tensor([[[ 0.4429, -0.4830,  0.3228,  0.6928],
         [ 0.5274, -0.0756,  0.8617, -0.2487]],

        [[ 0.3442,  0.5326,  0.3677,  0.8042],
         [ 0.0529,  0.5052, -0.0472,  0.4051]],

        [[ 0.4741,  0.1528,  0.2264,  0.2591],
         [ 0.3518, -0.1121,  0.3918,  0.1530]],

        [[ 0.4059,  0.4195, -0.0522,  0.4787],
         [ 0.3002,  0.6160,  0.2105,  0.7513]],

        [[ 0.2523,  0.0068,  0.4295,  0.1090],
         [ 0.4686,  0.1022,  0.2530,  0.2042]]], grad_fn=<StackBackward0>)
tensor([[[0.2523, 0.0068, 0.4295, 0.1090],
         [0.4686, 0.1022, 0.2530, 0.2042]]], grad_fn=<StackBackward0>)


In [587]:
inputs_stacked.shape

torch.Size([5, 2, 3])

In [541]:
hidden

(tensor([[[-0.1900,  0.0780,  0.2155]]], grad_fn=<StackBackward0>),
 tensor([[[-0.4337,  0.2092,  0.3789]]], grad_fn=<StackBackward0>))